In [1]:
import pandas as pd
import os
import re
import json

In [2]:
def dados_banco_dados():
    
    path_dados = '../Dados_originais/Dados_Banco_de_Dados'
    files_site = [os.path.join(path_dados, file) for file in os.listdir(path_dados)]
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('propost')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)


def dados_banco_dados_cep():
    
    path_dados = '../Dados_originais/Dados_Banco_de_Dados_CEP'
    files_site = [os.path.join(path_dados, file) for file in os.listdir(path_dados)]
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('propost')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)

def get_end_salvo(versao, path = None):
    
    if path is None:
        path = '../Dados_gerados/geocode'
    if os.path.exists(path):
        pat = 'enderecos_georref_\d*.json'
        files = [os.path.join(path, file) for file in os.listdir(path)
                if re.match(pat, file)]

        for f in files:
            versao_file = os.path.split(f)[-1].split('_')[-1].split('.')[0]
            if str(versao) == versao_file:
                with open(f) as file:
                    return json.load(file)

In [3]:
#pegando todos os dados para cruzar

contrib, prop = dados_banco_dados()
contrib_cep, prop_cep = dados_banco_dados_cep()
contrib_georref = pd.read_excel('../Dados_gerados/contrib_georref.xlsx')
prop_georref = pd.read_excel('../Dados_gerados/prop_georref.xlsx')
enderecos = get_end_salvo(versao = 1)

In [4]:
#CEP VEIO COMO NUMERO, tem que arrumar
contrib_cep.dtypes

ID                  int64
Numeração          object
Título             object
Texto              object
Opção escolhida    object
Sugestão           object
Usuário            object
Email              object
Endereço           object
CEP                 int64
dtype: object

In [5]:
def arrumar_cep(cep):
    
    if pd.isnull(cep):
        return None
    
    pat_cep = '^\d{5}-\d{3}'
    if re.match(pat_cep, str(cep)):
        return cep
    
    else:
        cep = str(int(cep))
        while len(cep) < 8:
            cep = '0' + cep
    
    return f'{cep[:5]}-{cep[5:]}'

In [6]:
contrib_cep['CEP'] = contrib_cep['CEP'].apply(arrumar_cep)
prop_cep['CEP'] = prop_cep['CEP'].apply(arrumar_cep)

In [7]:
#garantir que todos os dados originais vieram
assert prop['ID'].isin(prop_cep['ID']).all()
assert prop['ID'].isin(prop_georref['ID']).all()
assert prop_georref['ID'].isin(prop_cep['ID']).all()
assert contrib['ID'].isin(contrib_cep['ID']).all()
assert contrib['ID'].isin(contrib_georref['ID']).all()
assert contrib_georref['ID'].isin(contrib_cep['ID']).all()


In [8]:
#mesmo assim acho melhor continuar trabalhando com as bases originais
#entao vou fazer merge

contrib_cep = contrib_cep[['ID', 'CEP']].copy()
prop_cep = prop_cep[['ID', 'CEP']].copy()

contrib_georref_cep = pd.merge(contrib_georref, contrib_cep, how = 'left')#left para garantir
prop_georref_cep = pd.merge(prop_georref, prop_cep, how = 'left')

In [9]:
#garantir que todos os enderecos foram georreferenciados
assert prop_georref_cep[~prop_georref_cep['Endereco'].isin(enderecos)]['Endereco'].isnull().all()
assert contrib_georref_cep[~contrib_georref_cep['Endereço'].isin(enderecos)]['Endereço'].isnull().all()

In [10]:
prop_georref_cep.drop('Unnamed: 0', axis = 1, inplace = True)
contrib_georref_cep.drop('Unnamed: 0', axis = 1, inplace = True)

In [11]:
def get_cep_end(end, enderecos_totais):
    
    if pd.isnull(end):
        return None
    enderecos_encontrados = enderecos_totais[end]
    for endereco in enderecos_encontrados:
        for componente in endereco['address_components']:
            if 'postal_code' in componente['types']:
                return componente['long_name']
    
    return None

In [12]:
#pegando o CEP que o google devolveu
contrib_georref_cep['CEP_Google'] = contrib_georref_cep['Endereço'].apply(
    get_cep_end, args = [enderecos] )
prop_georref_cep['CEP_Google'] = prop_georref_cep['Endereco'].apply(
    get_cep_end, args = [enderecos] )

In [13]:
def check_cep(row):
    
    if pd.isnull(row['CEP']) and pd.isnull(row['CEP_Google']):
        return True
    else:
        check = row['CEP'] == row['CEP_Google']
        return check
    

In [14]:
comparacao = contrib_georref_cep['CEP_Google'] == contrib_georref_cep['CEP']
contrib_georref_cep['check_cep'] = comparacao

In [15]:
prop_georref_cep['check_cep'] = prop_georref_cep.apply(check_cep, axis = 1)
contrib_georref_cep['check_cep'] = contrib_georref_cep.apply(check_cep, axis = 1)

In [16]:
prop_georref_cep['check_cep'].sum()/len(prop_georref_cep)

0.8113425925925926

In [17]:
contrib_georref_cep['check_cep'].sum()/len(contrib_georref_cep)

0.9345670852610707

In [18]:
#salvando os datasets com a checagem
prop_georref_cep.to_excel('../Dados_gerados/prop_georref_cep.xlsx')
contrib_georref_cep.to_excel('../Dados_gerados/contrib_georref_cep.xlsx')

In [19]:
#identificando os endereços unicos nao georreferenciados
contrib_n_georref  = contrib_georref_cep[
    ~contrib_georref_cep['check_cep']]['endereço_original'].unique()
prop_n_georref = prop_georref_cep[
    ~prop_georref_cep['check_cep']]['endereço_original'].unique()

enderecos_n_georref = pd.concat([
    pd.Series(contrib_n_georref), pd.Series(prop_n_georref)
    ]).unique()

#tem que trocar o np.nan por None para o JSON
enderecos_n_georref = [item if not pd.isnull(item) else None
                       for item in enderecos_n_georref]

In [20]:
#salvando json
with open('../Dados_gerados/ends_n_georref.json', 'w') as f:
    json.dump(list(enderecos_n_georref), f)